In [1]:
%matplotlib widget
import numpy as np
import math
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

import random

import matplotlib.pyplot as plt

import re

from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook

from joblib import Parallel, delayed
import multiprocessing
from datetime import datetime

In [2]:
torch.set_printoptions(precision=10)

In [11]:
plikembd = open("../embeddings_small.txt")

embeddingsize = 400

embeddingscuda_chars = {}
embeddingscuda_chars['🈳'] = 0

embeddingscuda = torch.tensor([]).to("cuda:0")
embeddingscuda =  torch.cat((embeddingscuda, torch.zeros((embeddingsize)).to("cuda:0")), 0)

for cnt, line in enumerate(plikembd.readlines()):
    exp1 = line.replace("\n", "").split(":")
    
    znak = exp1[0]
    embedding = torch.tensor([float(liczba) for liczba in exp1[1].replace(" ", "").replace("[", "").replace("]", "").split(",\t")[:-1]]).to("cuda:0")
    
    embeddingscuda_chars[znak] = cnt+1
    embeddingscuda = torch.cat((embeddingscuda, embedding), 0)

embeddingscuda = embeddingscuda.reshape(-1, embeddingsize)

print(embeddingscuda.shape)
#print(embeddingscuda[embeddingscuda_chars['a']])

torch.Size([82, 400])


In [12]:
polskie = open("../100k.txt", encoding='utf8')
slowa = [slowo.replace("\n", "") for slowo in polskie.readlines()]
polskie.close()
print(slowa[:100])
print(len(slowa))
slowa = slowa[:13000]

['abmicro', '527579', 'agro2011', 'nakamichi', 'dugant', '29082009', 'hannah1', 'ma12po', '580813', 'lukasz4', 'kriket', 'werohaze', 'dron14121987', 'pinot1', 'maria59', 'moltrex', 'kilim', 'Zabeczka', '7937', '7maluszek', 'gacek16', 'ruleta1', '17Cyprian', 'maj2006', 'o2skarek', 'anno01', 'gusciol', 'gumis1', 'malaz11', 'watomzon', 'rumianek', 'neoisp', 'zsofie', 'zipera', '141084', 'ultraboy', '110567', 'anbog', 'troyd34', 'roxboro', 'werunia', 'misiokrzysio', 'hydra6', 'grzes10', 'kakigewy', 'filip98', '20021970', 'natka777', 'patricia', 'polisnova', 'miko20', 'prj2501', 'grobian1', '1972ml', 'robo21101967', 'jw17061958', 'gregordj', 'insko2000', 'maclear', 'zl060374', 'asia7', 'misiu777', 'varius2', 'lgflatron', 'rele1', '130178', 'centrala', 'filoman', '41karat2', '84jaysinner', 'agatina', 'ambasada', 'maly17', 'goszkam', '1branka', 'dasti', 'petro111', '070165', '20111965', 'triforce', 'bogumil1', 'anna3005', 'dron10011001', 'santaclara', 'pas2526', '11011975', 'rest12', 'j87neo'

In [13]:
longestword = 0

unikalneznaki = []
for slowo in slowa:
    for literka in slowo:
        if literka not in unikalneznaki:
            unikalneznaki.append(literka)
    if len(slowo) > longestword:
        longestword = len(slowo)
            
print(unikalneznaki)
print(longestword)
print(len(unikalneznaki))

['a', 'b', 'm', 'i', 'c', 'r', 'o', '5', '2', '7', '9', 'g', '0', '1', 'n', 'k', 'h', 'd', 'u', 't', '8', 'p', '3', 'l', 's', 'z', '4', 'e', 'w', 'x', 'Z', '6', 'C', 'y', 'j', 'f', 'v', 'F', 'P', 'R', 'O', 'T', 'A', 'X', 'L', 'E', 'S', 'I', 'K', 'N', 'D', 'M', 'q', 'H', 'U', 'J', 'Q', 'G', 'W', 'B', 'V', 'Y', '@', '.', '#', '!', ';', '*', '`']
15
69


In [14]:
print(list(embeddingscuda_chars.keys()))
print(len(list(embeddingscuda_chars.keys())))

['🈳', ' ', '!', '#', '$', '%', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ';', '=', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '^', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
82


In [16]:
print("ilosc wejsc: " + str(embeddingsize * longestword))
#print("ilosc wyjsc generatora: " + str(len(embeddingscuda[0]) * longestword))

ilosc wejsc: 6000


In [18]:
lstm = nn.LSTM(3, 3)
randomowe = torch.rand((2, 3, 3))

In [20]:
outs, hidden = lstm(randomowe)

In [21]:
outs

tensor([[[-0.1070592925, -0.0148423221,  0.1011685506],
         [-0.0487920418, -0.0193215031,  0.1305106878],
         [-0.0574714914, -0.0047326214,  0.1023265272]],

        [[-0.0547768213, -0.0078721950,  0.1857367456],
         [-0.0484669767,  0.0147769675,  0.1867975742],
         [-0.0411583595, -0.0205980875,  0.2024593949]]],
       grad_fn=<StackBackward>)

In [22]:
hidden

(tensor([[[-0.0547768213, -0.0078721950,  0.1857367456],
          [-0.0484669767,  0.0147769675,  0.1867975742],
          [-0.0411583595, -0.0205980875,  0.2024593949]]],
        grad_fn=<StackBackward>),
 tensor([[[-0.1031885445, -0.0235459059,  0.3336464763],
          [-0.0911864564,  0.0403079316,  0.3165190220],
          [-0.0753030553, -0.0624615289,  0.3600196838]]],
        grad_fn=<StackBackward>))

In [23]:
outs2, hidden2 = lstm(randomowe, hidden)

In [24]:
outs2

tensor([[[-1.3304784894e-01,  1.5914696269e-03,  2.4575383961e-01],
         [-6.5307952464e-02,  1.5198746696e-02,  2.8730246425e-01],
         [-7.5219586492e-02,  2.8761538488e-05,  2.6180109382e-01]],

        [[-5.6334562600e-02,  1.5697725117e-02,  2.9525142908e-01],
         [-4.5112848282e-02,  5.0680570304e-02,  3.0952945352e-01],
         [-4.1910879314e-02, -3.3875361551e-03,  3.2009175420e-01]]],
       grad_fn=<StackBackward>)

In [25]:
hidden2

(tensor([[[-0.0563345626,  0.0156977251,  0.2952514291],
          [-0.0451128483,  0.0506805703,  0.3095294535],
          [-0.0419108793, -0.0033875362,  0.3200917542]]],
        grad_fn=<StackBackward>),
 tensor([[[-0.1020751745,  0.0494741052,  0.5524598360],
          [-0.0813980550,  0.1468101740,  0.5451700091],
          [-0.0735868812, -0.0108587565,  0.5971056223]]],
        grad_fn=<StackBackward>))